In [0]:
import json
from glob import glob
from functions.sanity import validate_settings, initialize_empty_tables
from functions.utility import apply_job_type

# Load anything in layer_*_<color>/*.json
job_settings = {}
for color in [ 'bronze', 'silver', 'gold' ]:
    paths = glob(f'./layer_*_{color}/*.json')
    job_settings[color] = []
    for path in paths:
        table_name = path.split('/')[-1].split('.')[0]
        with open(path) as f:
            settings = json.load(f)
        settings = apply_job_type(settings)
        history = {
            'build_history': settings.get('build_history', 'false'),
            'history_schema': settings.get('history_schema'),
            'full_table_name': settings.get('dst_table_name'),
        }
        job_settings[color].append({'table': table_name, 'history': history})
for key, value in job_settings.items():
    dbutils.jobs.taskValues.set(key=key, value=value)

# Load anything in layer_*/*.json if it has a key 'dst_table_name'
history_settings = []
for p in glob('./layer_*/*.json'):
    with open(p) as f:
        settings = json.load(f)
        settings = apply_job_type(settings)
        if str(settings.get('build_history', 'false')).lower() == 'true':
            entry = {'full_table_name': settings['dst_table_name']}
            if 'history_schema' in settings:
                entry['history_schema'] = settings['history_schema']
            history_settings.append(entry)
dbutils.jobs.taskValues.set(key='history_settings', value=history_settings)

# Sanity check
validate_settings(dbutils)
initialize_empty_tables(spark)

